# Imports

In [1]:
#Needed for reading the data
import h5py as h5
#Needed for checking file existence
import os 
#Needd for array handling slicing
import numpy as np


# Custom imports (i.e. own scripts)

In [2]:
"""

If you use this ipython notebook outside its original directory
please set the path to the COMPAS repository and uncomment the lines below
this way it automatically finds the custom scripts that we need to import

"""
#pathCOMPASrepo       = '/home/cneijssel/Documents/COMPASpop/'
#pathToPostProcessing = 'popsynth/Papers/NeijsselEtAl/PostProcessing/1_RatePerUnitSolarMass/
#import sys
#sys.path.append('athCOMPASrepo+pathToPostProcessing)

#The script we use get the total amount of SFR
import totalMassEvolved  


# Why we want to know rates

As seen in

<a href="../2_BasicDataHandling/1_MakingAHistogram.html">
Reading a single column and making a histogram
</a>

We can get a distribution of a certain parameter and make for example a histogram.
In this case you simply have a total number of binaries or number of binaries in a bin.
The shape of these distributions is important to understand the physics in your model.
However to constrain a model we often look at observations.
Therefore, besides knowing the shape of a distribtion,
we want to know how often we expect to see a certain event.
In other words we want to know rates. In our case the rate depends on how often we form stars. 
Observations give us insight into the star formation rate. This quantity is usual defined as an amount of solar
mass per unit time or per unit time and volume that goes into the formation of all stars.
This means that if we know how much solar mass our simulation has in all stars we can use this observational input
to calculate the rate of events in our population.

Example: (random picked numbers) we get 1 double neutron star per 2 million solar masses evolved in all stars in our simulation. The galaxy has a star formation rate (SFR) of 2 solar masses per year. Then we expect to form
10^-6 binary black hole progenitors per year. This logic also applies to other events such as supernovae, mergers etc. Notice that we know the rate of forming the progenitors. The rate of observing the event depends on additional parameters (see for example cosmic integration).

We know the number of events now all we need to know is the amount of sollar masses evolved IN ALL STARS.
 

# Rate per unit solar mass evolved in COMPAS

In the COMPAS simulation we evolve a population of stars resulting in different types of events  (mergers, supernovae, common-envelopes, formation of binary-black-holes etc etc).

The initial parameters file tells us how much mass we evolved in all stars to get
a certain type of output.

Lets import the data and look at this quantity.

#### Path to the data

In [3]:
path     = '/home/cneijssel/Desktop/RLOF_test/First_Solar/'
filename = 'COMPASOutput.h5'

#### Define the h5file

In [4]:
if not  os.path.isfile(path+filename):
    raise ValueError("h5 file not found. Wrong path given?")
elif os.path.isfile(path+filename):
    Data  = h5.File(path+filename)

#### Get the total mass evolved in COMPAS

In [5]:
InitialParameters = Data['systems']

#This example simulation only has binaries
MassPrimaries     = InitialParameters['mass1'][...].squeeze()
MassSecondaries   = InitialParameters['mass2'][...].squeeze()
#Total mass per system in COMPAS
TotalMassSystems  = MassPrimaries + MassSecondaries
#Total mass in all of COMPAS
TotalMassCOMPAS   = np.sum(TotalMassSystems)
print('Total mass in stars evolved COMPAS =%s  [Msun]'%(TotalMassCOMPAS))

Total mass in stars evolved COMPAS =2030498.442539  [Msun]


In [6]:
#what are the number of double neutron stars?
DCOs         = Data['doubleCompactObjects']
#stellar type 13 = neutron star, below we mask double neutron stars
PrimaryNS    = DCOs['stellarType1'][...].squeeze() == 13
SecondaryNS  = DCOs['stellarType2'][...].squeeze() == 13
maskDNSs     = PrimaryNS & SecondaryNS
numberDNSs   = np.sum(maskDNSs)  #true counts as 1
print('We formed %s double neutron stars'%(numberDNSs))

We formed 150 double neutron stars


So now we know that we 150 double compact objects per 2 million solar masses evolved 
in the COMPAS simulation

However, the COMPAS simulation did not necessarily evolve all stars and therefore the 2 million solar
masses does not relate to the star formation rate.
For example, if we care about forming double neutron stars in our simulation we dont evolve stars
with masses below 6~7 solar masses nor do we evolve single stars. We could but we would be wasting computational time. What we need to recover instead is the amount of solar masses that COMPAS would have evolved in all stars to get the same result.

Using some assumptions and knowledge of our initial distributions we can recover this using 
the totalMassEvolved script. This returns the approximated fraction of the total star forming mass that COMPAS
simulated. More details/assumptions we leave for other notes, here we show the basic.

We need to know the lower and upper mass limit of our simulation and the assumed fraction of stars in binaries

In [7]:
lowerMassPrimary = 5.
upperMassPrimary = 150
binaryFraction   = 0.7

fraction = totalMassEvolved.\
          fractionTotalMassEvolved(Mlower=lowerMassPrimary,\
                                   Mupper=upperMassPrimary,\
                                   binaryFraction=binaryFraction)
    
print('Total mass in stars evolved COMPAS =%s  [Msun]'%(TotalMassCOMPAS))
print('Fraction of all mass evolved in COMPAS = %s '%(fraction))
totalStarFormingMass = TotalMassCOMPAS/float(fraction)
print('Total mass in all stars is %s'%(totalStarFormingMass))
RatePerMsun = numberDNSs / float(totalStarFormingMass)
print('Number of double neutron stars per unit star formation  =%s  [Msun^-1]'%(RatePerMsun))

Total mass in stars evolved COMPAS =2030498.442539  [Msun]
Fraction of all mass evolved in COMPAS = 0.21078317511297273 
Total mass in all stars is 9633114.414614547
Number of double neutron stars per unit star formation  =1.5571288115547832e-05  [Msun^-1]


This normalisation can also be used in histograms to give the distribution per unit SFR in etc

In [8]:
#I am done here see ya!
Data.close()